In [1]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib

In [2]:
from fastai.tabular.all import *
from pathlib import Path

In [3]:
modelFolder = Path.cwd().parent.parent / 'TrainedModels'
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
animalClassifier.pkl
countryGuesser.pkl
leagueChampionClassifier.pkl
politicianClassifier.pkl
stockScreenerV1.pkl
stockScreenerV2.pkl
stockScreenerV3.pkl
stockScreenerV4.pkl
stockScreenerV5.pkl
test.pkl


In [4]:
modelName = 'stockScreenerV5.pkl'
learn = load_learner(modelFolder / modelName)

In [5]:
from ipynb.fs.full.stockFetcher import getTickerData

In [63]:
predictionTarget = 'SONY'

test_df = getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
	test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)
test_df.head()

c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Industry
0,20.610001,20.700001,20.540001,20.645,2786020,0.0,0.0,1.725505,1.243898e+11,0.457837,Consumer Electronics


In [64]:
prediction = learn.get_preds(dl=dl)
print(f"Prediction for {predictionTarget}:")
print(f"{prediction[0][0][0].item() * 100:.2f}%")

Prediction for SONY:
79.52%
